From: Modular examples/Mandelbrot.ipynb, But using Moplex Hybrid Numbers.

# Import moplex and utils

In [ ]:
import benchmark
from math import iota
from sys import num_physical_cores
from algorithm import parallelize, vectorize
from src import HybridSIMD
from python import Python

alias type = DType.float64
alias Hybrid = HybridSIMD[type=type]

alias MAX_ITERS = 1000
alias ESCAPE = 1000

alias width = 800
alias height = 800

alias min_x = -2.0
alias max_x = 2.0
alias min_y = -2.0
alias max_y = 2.0


# Install python dependencies if missing

In [ ]:
%%python
from importlib.util import find_spec
import shutil
import subprocess

def install_if_missing(name: str):
    if find_spec(name):
        return
    print("missing", name)
    print(f"{name} not found, installing...")
    try:
        if shutil.which('python3'): python = "python3"
        elif shutil.which('python'): python = "python"
        else: raise ("python not on path" + fix)
        subprocess.check_call([python, "-m", "pip", "install", name])
    except:
        raise ImportError(f"{name} not found" + fix)

install_if_missing("numpy")
install_if_missing("matplotlib")

# Define matrix for storing results

In [ ]:
@value
struct Matrix[type: DType, rows: Int, cols: Int]:
    var data: UnsafePointer[Scalar[type]]

    fn __init__(inout self):
        self.data = UnsafePointer[Scalar[type]].alloc(rows * cols)

    fn __getitem__(self, row: Int, col: Int) -> Scalar[type]:
        return self.data.load(row * cols + col)

    fn store[width: Int = 1](self, row: Int, col: Int, val: SIMD[type, width]):
        self.data.store[width=width](row * cols + col, val)

# Define a function for visualizing results

In [ ]:
def show_plot[type: DType](matrix: Matrix[type, height, width]):
    alias scale = 1
    alias dpi = 1

    np = Python.import_module("numpy")
    plt = Python.import_module("matplotlib.pyplot")
    colors = Python.import_module("matplotlib.colors")

    numpy_array = np.zeros((height, width), np.float64)

    for row in range(height):
        for col in range(width):
            numpy_array.itemset((row, col), matrix[row, col])

    fig = plt.figure(1, [width, height], dpi)
    ax = fig.add_axes([0.0, 0.0, 1.0, 1.0], False, 1)
    light = colors.LightSource(315, 10, 0, 1, 1, 0)

    image = light.shade(numpy_array, plt.cm.hot, colors.PowerNorm(0.3), "hsv", 0, 0, 1.5)
    plt.imshow(image)
    plt.axis("off")
    plt.show()

# Mandelbrot kernel

In [ ]:
fn mandelbrot_kernel(c: Hybrid) -> SIMD[DType.index, c.size]:
    """A vectorized implementation of the inner mandelbrot computation."""
    var h = c
    var iters = SIMD[DType.index, c.size](0)

    var t: SIMD[DType.bool, c.size] = True
    for _ in range(MAX_ITERS):
        if not any(t):
            break
        h = h * h + c
        t = abs(h.denomer()) <= ESCAPE
        iters = t.select(iters + 1, iters)
    return iters

# Run mandelbrot and display results

In [ ]:
fn run_mandelbrot() raises:
    var matrix = Matrix[DType.index, height, width]()

    @parameter
    fn mandelbrot_worker(row: Int):
        alias scale_x = (max_x - min_x) / width
        alias scale_y = (max_y - min_y) / height

        @parameter
        fn compute_vector[simd_width: Int](col: Int):
            """Each time we operate on a `simd_width` vector of pixels."""
            var cx = min_x + (col + iota[type, simd_width]()) * scale_x
            var cy = min_y + row * SIMD[type, simd_width](scale_y)
            var c = Hybrid[simd_width, 1](cx, cy)
            matrix.store(row, col, mandelbrot_kernel(c))

        # Vectorize the call to compute_vector where call gets a chunk of pixels.
        vectorize[compute_vector, simdwidthof[type]()](width)


    @parameter
    fn bench():
        parallelize[mandelbrot_worker](height, height)

    var time = benchmark.run[bench](max_runtime_secs=0.5).mean("ms")

    show_plot(matrix)
    matrix.data.free()
    print(time)

run_mandelbrot()